### Compare two training runs against each other using Pytorch-FID package instead of built-in FID from SG2Ada

- This is done to establish the baseline FID using this FID implementation and compare with FID of various inversion methods later

In [1]:
import os
import pickle
from tqdm import tqdm
import torch
from PIL import Image

In [2]:
import platform
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

### Specify Model Paths

In [3]:
experiments_path = f"{DATA_PATH}/Models/Stylegan2_Ada/Experiments"
model1 = f"{experiments_path}/00003-stylegan2_ada_images-mirror-auto2-kimg1000-resumeffhq512/network-snapshot-000920.pkl"
model2 = f"{experiments_path}/00005-stylegan2_ada_images-mirror-auto2-kimg5000-resumeffhq512/network-snapshot-001200.pkl"

# Specify paths to save generated images
out_dir = f"{DATA_PATH}/Generated_Images/SG2Ada"
out1 = f"{out_dir}/00003_snapshot_920/"
out2 = f"{out_dir}/00005_snapshot_1200/"

### Load Models

In [4]:
os.chdir(f"{ROOT_PATH}/stylegan2-ada-pytorch/")

In [5]:
try:
    if torch.cuda.is_available():
        DEVICE = 'cuda'
    elif torch.backends.mps.is_available():
        DEVICE = 'mps'
    else:
        DEVICE = 'cpu'
except:
    if torch.cuda.is_available():
        DEVICE = 'cuda'
    else:
        DEVICE = 'cpu'
print(f"Using {DEVICE} as device")

with open(model1, 'rb') as f:
    architecture = pickle.load(f)
    G1 = architecture['G_ema'].to(DEVICE)
with open(model2, 'rb') as f:
    architecture = pickle.load(f)
    G2 = architecture['G_ema'].to(DEVICE)

Using cuda as device


### Generate Random Images

In [6]:
def generate_random(G, out_path, num_img):
    for i in tqdm(range(num_img)):
        z = torch.randn([1, G.z_dim]).to(DEVICE)
        c = None  
        img = G(z, c,  force_fp32=True, noise_mode = 'const')
        img_perm = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        output = Image.fromarray(img_perm[0].cpu().numpy(), 'RGB')
        output.save(f"{out_path}{i}.jpg")

In [7]:
num_samples = 50000

if not os.path.exists(out1):
    os.mkdir(out1)
    generate_random(G1, out1, num_samples)
if not os.path.exists(out2):
    os.mkdir(out2)
    generate_random(G2, out2, num_samples)

### Calculate FID Features and save for later use

In [8]:
# Calculate features used in FID for real dataset and save for later use
path_to_real = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/"
save_path_features = f"{DATA_PATH}/Metrics/FID/features_real_dataset.npz"
# Save Features of original dataset
if not os.path.exists(save_path_features):
    CMD = f"python -m pytorch_fid --save-stats {path_to_real} {save_path_features} --device cuda:0"
    !{CMD}
else:
    print("Features already calculated, ready to use")


Features already calculated, ready to use


In [9]:
# Calculate features used in FID for generated dataset and save for later use
save_path_features_00003 = f"{DATA_PATH}/Metrics/FID/features_generated_00003.npz"
# Save Features of original dataset
if not os.path.exists(save_path_features_00003):
    CMD = f"python -m pytorch_fid --save-stats {out1} {save_path_features_00003} --device cuda:0"
    !{CMD}
else:
    print("Features already calculated, ready to use")

Features already calculated, ready to use


In [10]:
# Calculate features used in FID for generated dataset and save for later use
save_path_features_00005 = f"{DATA_PATH}/Metrics/FID/features_generated_00005.npz"
# Save Features of original dataset
if not os.path.exists(save_path_features_00005):
    CMD = f"python -m pytorch_fid --save-stats {out2} {save_path_features_00005} --device cuda:0"
    !{CMD}
else:
    print("Features already calculated, ready to use")

Features already calculated, ready to use


### Calculate FID

In [11]:
# Calculate FID for model 1
CMD = f"python -m pytorch_fid {save_path_features} {save_path_features_00003}"
!{CMD}

FID:  10.333990323969601


In [12]:
# Calculate FID for model 2
CMD = f"python -m pytorch_fid {save_path_features} {save_path_features_00005}"
!{CMD}

FID:  8.0966246619368
